# Porting LSST test queries to Stripe82++ : part 2
Ani will do this one.

Notebook with queries representing LSST test queries, but against the Stripe82 database. The latter was extended with a 'match' and 'coadd' table following script in 
https://docs.google.com/document/d/1qFOE1s4bQUDM8W7wLzoxb2OK1LaPwzd5Lmy_GKUYut0.

Queries from https://dev.lsstcorp.org/trac/wiki/db/queries, second batch:
<pre>
Cone-magnitude-color search 
Select all galaxies in given area 
Select all variable objects in given area 
Select time series data for all objects in a given area of the sky, in a given photometric band with a given variability index 
How many objects cross both Jupiter and Saturn? 
Select all unresolved objects (stellar PSF) in a given region of sky, and return their attributes 
Select all unresolved object in a specified region with cuts on attributes 
For a specified patch of sky, give me the source count density of unresolved sources (star like morphology) 
</pre>

Assuming LSST schema S12_sdss from https://lsst-web.ncsa.illinois.edu/schema/index.php?sVer=S12_sdss

Ths notebook is in http://github/com/idies/jhu-lsst/Stripe82/test_queries/LSST-Test-Queries%20-%20Part%202.ipynb

In [ ]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
import skimage.io                   # image processing library


import SciServer.CasJobs as cj
import SciServer.SkyServer as skyS

In [1]:
# define some variable to be used in query execution
TARGET_CONTEXT="Stripe82_016"
raMin = "308"
raMax = "310"
decMin = "-1.5"
decMax = "-0.5"
zMin = "19"
zMax ="22"
grMin = "1"
grMax = "1.5"
izMin = "0.5"
izMax = "1"

### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/003" target="_blank">Cone-magnitude-color search</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/003

QSERV
```sql
SELECT  *
FROM    Object
WHERE   areaSpec_box(:raMin, :declMin, :raMax, :declMax)
   AND  zMag      BETWEEN :zMin  AND :zMax
   AND  gMag-rMag BETWEEN :grMin AND :grMax
   AND  iMag-zMag BETWEEN :izMin AND :izMax
```

In [ ]:
# Replaced the "select *" with a list to avoid a huge output.  Also added a "top 10". The HTM version is not working ATM,
# looks like the assembly is not initialized properly in the DB.
sql="""
SELECT  TOP 10 b.objID, b.ra, b.dec, b.u, b.g, b.r, b.i, b.z
FROM 
   -- dbo.fGetObjFromRectEq( 180, -0.5, 180.1, -0.4) a      -- HTM version
   -- JOIN PhotoObjAll b ON a.objID=b.objID                 -- HTM Version
   PhotoObjAll b 
WHERE
   ra between """+raMin+""" and """+raMax+"""
   AND dec between """+decMin+""" and """+decMax+"""
   AND  z BETWEEN """+zMin+""" and """+zMax+"""
   AND  g-r BETWEEN """+grMin+""" and """+grMax+"""
   AND  i-z BETWEEN """+izMin+""" and """+izMax+"""
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/014" target="_blank">Select all galaxies in given area</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/014
    
QSERV
```sql
SELECT objectId
FROM   Object
WHERE  areaSpec_box(:raMin, :declMin, :raMax, :declMax)
AND    extendedParameter > 0.8
```

In [16]:
# Replaced "extendedParameter" with Petrosian radius in r band and ran it on Galaxy view.  
# Also added a "top 10". The HTM version is not working ATM, looks like the assembly is not initialized properly in the DB.
rMin = "2.5"
sql="""
SELECT  TOP 10 b.objID, b.ra, b.dec, b.petroRad_r
FROM 
   -- dbo.fGetObjFromRectEq( 180, -0.5, 180.1, -0.4) a      -- HTM version
   -- JOIN PhotoObjAll b ON a.objID=b.objID                 -- HTM Version
   Galaxy b 
WHERE
   ra between """+raMin+""" and """+raMax+"""
   AND dec between """+decMin+""" and """+decMax+"""
   AND petroRad_r > """+rMin+"""
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

,objID,ra,dec,petroRad_r
0,8658194323094438011,308.000001,-1.080312,2.969953
1,8658200920164206050,308.000001,-1.219354,2.729609
2,8658476657062119693,308.000002,-1.227750,4.583994
3,8658174750926046477,308.000002,-1.219350,4.738014
4,8658198588533179532,308.000003,-0.733367,2.970514
5,8658197844966310164,308.000003,-1.057900,2.969805
6,8658188335907932731,308.000004,-1.227764,2.970030
7,8658197973817952061,308.000006,-1.203791,2.969872
8,8658197879326378193,308.000010,-1.203811,2.969768
9,8658198064010494056,308.000013,-1.086952,16.800340


### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/008" target="_blank">Select all variable objects in given area</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/008

QSERV
```sql
SELECT objectId
FROM   Object
WHERE  areaSpec(:raMin, :declMin, :raMax, :declMax)
AND    variability > 0.8
```

In [11]:
# Need to find a substitute for the "variability" column that is used in the source query (but doesn't exist in their
# schema either).
sql="""
SELECT  TOP 10 p.objID
FROM 
   -- dbo.fGetObjFromRectEq( 180, -0.5, 180.1, -0.4) o      -- HTM version
   -- JOIN PhotoTag p ON o.objID=p.objID                    -- HTM Version
   PhotoTag p 
WHERE
   p.ra between """+raMin+""" and """+raMax+"""
   AND p.dec between """+decMin+""" and """+decMax+"""
-- need some kind of variability test:
--  AND variabiliy > 0.8
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

,objID
0,8658194658100054697
1,8658194323094438011
2,8658201040424077601
3,8658198235809187648
4,8658476657062118983
5,8658200567976363654
6,8658200920164206050
7,8658476657062119693
8,8658200950228648999
9,8658197999585921342


### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/007" target="_blank">Select time series data for all objects in a given area of the sky, in a given photometric band with a given variability index</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/007

QSERV
```sql
SELECT objectId, taiMidPoint, fluxToAbMag(psfMag)
FROM   Source
JOIN   Object USING(objectId)
JOIN   Filter USING(filterId)
WHERE  areaSpec_box(:raMin, :declMin, :raMax, :declMax)
  AND  filterName = 'u'
  AND  variability BETWEEN :varMin AND :varMax
ORDER BY objectId, taiMidPoint ASC
```

In [ ]:
# Need a variability column for this one too. Not sure if it variability is meant to be mmeasured per filter (their page says
# the column should be in Object, not Source, so presumably not). Anyway, tried a naive variability measure using stdev of 
# psfMag in r band.
sql="""
SELECT 
    TOP 10
    p.objID, f.mjd_u, p.psfMag_u, a.sigma
FROM MatchHead h
    JOIN PhotoTag p ON p.objID=h.objID
    CROSS APPLY (SELECT STDEV(q.r) AS sigma 
                 FROM Match m 
                     JOIN Photoobj q ON m.objID2=q.objID 
                 WHERE m.objID1=h.objID AND q.r BETWEEN 0 AND 25) a    
    JOIN Field f ON f.fieldID=p.fieldID
WHERE 
   p.ra between """+raMin+""" and """+raMax+"""
   AND p.dec between """+decMin+""" and """+decMax+"""
   AND a.sigma > 1
ORDER BY p.objID, f.mjd_u ASC
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df

### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/050" target="_blank">How many objects cross both Jupiter and Saturn?</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/050
    
QSERV
```sql
SELECT COUNT(movingObjectId) 
FROM MovingObject 
WHERE q < 5.2             -- perihelion distance
  AND q*(1+e)/(1-e) > 9.6 -- aphelion distance
--     note for some objects this may not be well defined (e may be very close to 1) 
```

In [ ]:
# Not sure how we can do this one at the moment.

### <a href="https://dev.lsstcorp.org/trac/wiki/db/queries/050" target="_blank">Select all unresolved objects (stellar PSF) in a given region of sky, and return their attributes</a>

https://dev.lsstcorp.org/trac/wiki/db/queries/053

QSERV
```sql
SELECT parallax, uMag, gMag, rMag, iMag, zMag, yMag, muRA, muDecl,
       uVarProb, gVarProb, rVarProb, iVarProb, zVarProb, yVarProb
FROM   Object
WHERE  areaSpec(:raMin, :declMin, :raMax, :declMax)
  AND  extendedParameter < 0.2
```

In [ ]:
# Do we have parallax in the Stripe 82 db?  Looks like ProperMotions table is empty. Otherwise this query is simpler in SDSS.
sql="""
SELECT 
-- parallax, 
    u, g, r, i, z, 
-- muRA, muDecl,
-- uVarProb, gVarProb, rVarProb, iVarProb, zVarProb, yVarProb
FROM   Star
WHERE  
   p.ra between """+raMin+""" and """+raMax+"""
   AND p.dec between """+decMin+""" and """+decMax+"""
"""
df=cj.executeQuery(sql,TARGET_CONTEXT)
#g=pandas.read_csv(gals)
df